## Домашняя работа #5.

**Kaggle: [competition](https://www.kaggle.com/competitions/hse-2023-hw-5), [invite link](https://www.kaggle.com/t/6d51db6f2dde497a9c12ecf28899082f)**

### Описание

Вам предоставлен измененный датасет CIFAR10. В нём содержится 50+10 тысяч RGB изображений размера 32х32 следующих 10 классов: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

Задача: используя свёрточные нейронные сети, добиться максимальной точности классификации.

### Данные

В данном ноутбуке уже есть код для PyTorch, отвечающий за загрузку данных. Если будете использовать этот фреймворк, можно пропустить секцию. Если же хочется использовать другой фреймворк глубокого обучения, ниже дано краткое описание формата хранения данных.

Каждый из файлов — сериализованный с помощью pickle c 4ой версией протолока python-словарик.

Список файлов:

- `meta` — метаданные датасета (например, названия классов)
- `data_train` — данные обучения, 50к примеров, по 5к на один класс
- `data_test` — данные теста, 10к примеров, по 1к на один класс, без ground-truth классов

Словарики с данными имеют следующие поля:

- `section` — имя части данных (обучение/тест)
- `names` — хеш-идентификаторы объекта
- `labels` — ground-truth классы, список из N чисел от 0 до 9
- `images` — numpy массив размером `(N, 3*32*32)` с изображениями

### Оценка

Качество решения будет оцениваться по метрике "точность". Точность – это количество правильно классифицированных картинок к общему числу картинок в тестовом наборе. Публичный лидерборд рассчитывается по 30% тестовых данных, поэтому старайтесь не переобучаться под него.

```
accuracy = (correct classified) / (total # of examples)
```

В качестве решения вы должны прислать файл формата:

```
Id,Category
0, 3
1, 2
2, 9
3, 1
...
```

где:

- `Id` — порядковый номер объекта в тестовом датасете
- `Category` — предсказанный класс объекта

В данном ноутбуке уже есть код, подготавливающий файл решения.

Итоговая оценка складывается из двух:
- по 5 баллов за преодоление каждого из бенчмарков (от 0 до 15 баллов)
- от 0 до 15 баллов за сам код решения

### Эксперименты

Так как в обучении нейросетевой модели есть очень много различных гиперпараметров, в данном домашнем задании нужно будет делать много различных экспериментов. И будет очень полезно, и для вас самих, и для нас, проверяющих, наличие текстового описания всех, или по крайней мере самых интересных / важных в выборе итоговой модели, экспериментов с результатами в финальном ноутбуке. Наличие и подробность описания экспериментов будут входить в итоговую оценку за домашнее задание. Дедлайн на Kaggle специально поставлен немного раньше, чтобы у вас было время спокойно дописать отчет об экспериментах.

## Решение

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from IPython.display import clear_output

In [ ]:
import os
import pickle
from typing import Any, Callable, Optional, Tuple
from PIL import Image

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor, Compose
from torchvision.datasets.vision import VisionDataset

Данные лежат в секции [Data](https://www.kaggle.com/c/csc-cv21-hw5/data) kaggle-соревнования. <br>
Нужно скачать архив с данными и расспаковать его. <br>
В переменной ниже надо указать путь до датасета. <br>

In [ ]:
dataset_root = "data/"

Код для загрузки измененного датасета CIFAR10. Аргументы инициализации:

- `root` — строка, путь до директории с файлами датасета
- `train` — флаг, загружать часть для обучения или теста
- `transform` — преобразования изображения
- `target_transform` — преобразования класса изображения

In [ ]:
class CIFAR10(VisionDataset):

    def __init__(self,
                 root: str,
                 train: bool = True,
                 transform: Optional[Callable] = None,
                 target_transform: Optional[Callable] = None,
                 ) -> None:

        super().__init__(root, transform=transform, target_transform=target_transform)
        self.train = train

        meta_path = os.path.join(self.root, 'meta')
        with open(meta_path, "rb") as f:
            content = pickle.load(f)
            self.classes = content['label_names']
            self.class_to_idx = {_class: i for i, _class in enumerate(self.classes)}

        data_path = os.path.join(self.root, 'data_train' if train else 'data_test')
        with open(data_path, "rb") as f:
            content = pickle.load(f)
            self.data = content['images'].reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))
            self.targets = content.get('labels')

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        img = Image.fromarray(self.data[index])
        target = self.targets[index] if self.targets else len(self.classes)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target

    def __len__(self) -> int:
        return len(self.data)

    def extra_repr(self) -> str:
        split = "Train" if self.train is True else "Test"
        return f"Split: {split}"

### Загрузка датасета

Загружаем часть датасета для обучения.

In [ ]:
data = CIFAR10(
    root=dataset_root,
    train=True,
    transform=ToTensor(),
)

Разбиваем случайным образом датасет на обучение и валидацию. <br>
На первой части будем обучать модель классификации. <br>
На второй части будем оценивать качество во время экспериментов. <br>

In [ ]:
train_data, val_data = torch.utils.data.random_split(
    data,
    [40000, 10000],
    generator=torch.Generator().manual_seed(42),
)

Инициализируем data loader-ы.

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_data, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

Посмотрим, какой размерности батчи выдает data loader.

In [ ]:
for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

### Модель классификации

Определяем, на каком устройстве будем обучать модель.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


Train-test loops

In [ ]:
import IPython
from math import ceil
from tqdm import tqdm

def train_loop(model, dataloader, loss_fn, optimizer, step=0.05, history_loss=None, history_acc=None):
    out = display(IPython.display.Pretty('Learning...'), display_id=True)

    size = len(dataloader.dataset)
    len_size = len(str(size))
    batches = ceil(size / dataloader.batch_size) - 1

    train_acc, train_loss = [], []
    percentage = 0

    for batch, (X, y) in enumerate(tqdm(dataloader, leave=False, desc="Batch #")):
        X, y = X.to(device), y.to(device)

        pred=model.forward(X)
        loss=loss_fn(pred,y)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch / batches > percentage or batch == batches:
            out.update(f'[{int(percentage * size)}/{size}] Loss: {loss:>8f}')
            percentage += step

    if history_loss is not None:
        history_loss.append(np.mean(train_loss))
    if history_acc is not None:
        history_acc.append(np.mean(train_acc))

    return {'train_loss': np.mean(train_loss), 'train_acc': np.mean(train_acc)} #TO DO

def test_loop(model, dataloader, loss_fn, history_loss=None, history_acc=None):

    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    batches = ceil(size / dataloader.batch_size)

    val_loss, val_acc = [], []

    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device), y.to(device)
            pred=model.forward(X)
            cur_loss=loss_fn(pred,y).item()
            test_loss+=cur_loss
            val_loss.append(cur_loss)

            cur_cor=(pred.argmax(1) == y).type(torch.float).sum().item()
            correct+=cur_cor
            val_acc.append(cur_cor)

    test_loss /= batches
    correct /= size

    print(f"Validation accuracy: {(100*correct):>0.1f}%, Validation loss: {test_loss:>8f} \n")

    if history_loss is not None:
        history_loss.append(np.mean(val_loss))
    if history_acc is not None:
        history_acc.append(np.mean(val_acc))

    return {'val_loss': np.mean(val_loss), 'val_acc': np.mean(val_acc)}


Задаем архитектуру модели классификации. <br>
Тут большой простор для разных экспериментов. <br>

Сначала я опробовал ResNet18. Реализацию которого я уже делал в процессе другого курса.

In [ ]:
class ResidualBlock(nn.Module):
     def __init__(self, in_channels, out_channels):
         super().__init__()
         stride = (2, 2) if in_channels != out_channels else (1, 1)

         self.shortcut=nn.Sequential(
              nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1),stride=stride, bias=False),
              nn.BatchNorm2d(out_channels)
         )

         self.activation=nn.ReLU()
         self.conv1=nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                kernel_size=(3, 3), padding=(1, 1), stride=stride, bias=False)
         self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels,
                                kernel_size=(3, 3), padding=(1, 1), stride=(1, 1), bias=False)
         self.bn1 = nn.BatchNorm2d(num_features=out_channels)
         self.bn2 = nn.BatchNorm2d(num_features=out_channels)


     def forward(self, x):

         residual = self.shortcut(x)
         x = self.activation(self.bn1(self.conv1(x)))
         x = self.bn2(self.conv2(x))

         return x + residual


class ResNetLayer(nn.Module):

     def __init__(self, in_channels, out_channels):
         super().__init__()
         self.blocks = nn.Sequential(ResidualBlock(in_channels, out_channels),
                                     ResidualBlock(out_channels, out_channels))


     def forward(self, x):
         x = self.blocks(x)
         return x


class ResNet18(nn.Module):
     def __init__(self, in_channels=3, n_classes=10):
         super().__init__()
         self.conv1=nn.Conv2d(in_channels, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         self.conv2=nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         self.conv3=nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

         self.bn1 = nn.BatchNorm2d(64)
         self.activation = nn.ReLU()

         self.layers = nn.Sequential(
                     ResNetLayer(64, 64),
                     ResNetLayer(64, 128),
                     ResNetLayer(128, 256),
                     ResNetLayer(256, 512),
                )
         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
         self.fc = nn.Linear(512, n_classes)
         self.LogSoftmax = nn.LogSoftmax()


     def forward(self, x):
         x = self.maxpool(self.conv3(self.conv2(self.conv1(x))))
         x = self.layers(x)
         x = self.avgpool(x)
         x = x.view(x.size(0), -1)
         x = self.LogSoftmax(self.fc(x))


         return x

In [ ]:
model = ResNet18().to(device)

В результате обучения на 30 эпохах с оптимизатором Adam и кросс-энтропией в качестве функции потерь, итоговая точность (accuracy) составила около 80%

Далее я стал использовать предобученные сетки. На них (да и в целом нужно), важно использовать предобработку изображений, как минимум рескейл в тот размер, на котором училась сетка, а также нормализацию.

Я пробовал VGG-16BN и разные версии EfficientNet. Суть работы над ними была одинаковая, далее расскажу про EffNet, так как на нем получился итоговый результат.

В качетсве трансформации использовалась функция *torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1.transforms()*,
благодаря которой изображение ресайзится до размера 256, потом применяется кроп размера 224, а затем изображение нормализуется со средними (0.485, 0.456, 0.406) and стандартными отклонениями (0.229, 0.224, 0.225). Если посчитать по датасету, то получаются примерно такие же значения.



В итоге бралась предобученная модель и менялась только лишь голова -- последний слой. Обучался лишь он.

In [ ]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

eff_net = efficientnet_b0(weights=EfficientNet_B0_Weights)

for param in eff_net.parameters():
    param.requires_grad = False


eff_net.classifier[1]=nn.Linear(in_features=1280, out_features=10)

Такой подход не позволил добится точности выше 80%. Поэтому далее попробовал обучать всю сетку, но с низким learning rate = 1e-5

In [ ]:
eff_net = efficientnet_b0(weights=EfficientNet_B0_Weights)

eff_net.classifier[1]=nn.Linear(in_features=1280, out_features=10)
eff_net = eff_net.to(device)

optimized_params = []
for param in eff_net.parameters():
    if param.requires_grad:
        optimized_params.append(param)

optimizer = torch.optim.Adam(optimized_params, lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(model, train_dataloader, loss_fn, optimizer)
    test_loop(model, test_dataloader, loss_fn)
    torch.save(model.state_dict(), 'model.pth')

В результате точность уже на третей эпохе достигла 0.95, чего было для меня достаточно (оставалось мало времени, да и так достаточно близко к точности человека)

### Отправка решения

Загружаем часть датасета для теста.

In [ ]:
test_data = CIFAR10(
    root=dataset_root,
    train=False,
    transform=ToTensor(),
)

In [ ]:
test_dataloader = DataLoader(
    test_data,
    batch_size=batch_size,
)

Делаем предсказания итоговой моделью.

In [ ]:
predictions = []

model.eval()
with torch.no_grad():
    for X, _ in test_dataloader:
        X = X.to(device)
        pred = model(X).argmax(1).cpu().numpy()
        predictions.extend(list(pred))

Формируем файл решения для отправки в kaggle.

In [ ]:
def write_solution(filename, labels):
    with open(filename, 'w') as solution:
        print('Id,Category', file=solution)
        for i, label in enumerate(labels):
            print(f'{i},{label}', file=solution)

write_solution('solution.csv', predictions)